# Evaluating Point-cloud Object Detections with FiftyOne

Point-clouds are an efficient way of representing three dimensional spatial data. They can be generated via either laser scanning techniques, such as [Lidar](https://en.wikipedia.org/wiki/Lidar), or photogrammetry, and are used in a variety of computer vision arenas, including autonomous vehicles, robots, and augmented and virtual reality. 

This walkthrough demonstrates how to use FiftyOne to work with three dimensional point-cloud data. 

In the walkthrough, we will cover the following topics:

* Loading a dataset with point-cloud data into FiftyOne
* Visualizing three dimensional data with the FiftyOne $3d$ visualizer
* Generating predictions from point-cloud data.
* Evaluating three-dimensional object detections.

#### So, what's the takeaway?

While the preprocessing and post-processing of point-cloud data can be quite distinct from the analogous image tasks, the same (or very similar) evaluation and visualization techniques apply.

## Setup

If you haven’t already, install FiftyOne:

In [2]:
# !pip install fiftyone

In this tutorial, we’ll use a Complex-YOLO model in the [FiftyOne Model Zoo](https://voxel51.com/docs/fiftyone/user_guide/model_zoo/index.html). This model is implemented in PyTorch, and to use it, you’ll need to have `torch` and `torchvision` installed.

In [3]:
# !pip install torch torchvision

Now we import all of the packages we will be using:

In [4]:
import math
import numpy as np
import cv2
import torch
import open3d as o3d
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.utils.kitti as fouk
import fiftyone.utils.utils_3d as fou3d
from fiftyone import ViewField as F

The data we will be working with is the [KITTI Multiview dataset](https://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=3d), which is conveniently available for download via the [FiftyOne Dataset Zoo](https://voxel51.com/docs/fiftyone/user_guide/dataset_zoo/index.html).  

**NOTE**: In order to robustly capture a three dimensional scene, an individual point-cloud can have hundreds of thousands, millions, or even tens of millions of points. As a result, three dimensional dataset can be quite large. The KITTI Multiview dataset is no exception. It takes up $53.34$GB. All but one section of this tutorial (projecting $3d$ detections into $2d$) work with the [Quickstart Groups dataset](https://voxel51.com/docs/fiftyone/user_guide/dataset_zoo/datasets.html#quickstart-groups), which only requires $516.3$MB of storage. If you want to use the quickstart data instead, switch which of the two lines below is commented out.

Here, we are only going to use the training split of the dataset.

In [7]:
dataset = foz.load_zoo_dataset(
    "kitti-multiview",
    split="train",
    dataset_name="pcd_tutorial"
)
                              
# dataset = foz.load_zoo_dataset(
#     "quickstart-groups",
#     split="train",
#     dataset_name="pcd_tutorial"
# )

Parsing dataset metadata
Found 22443 samples
Dataset info written to '/Users/jacobmarks/fiftyone/kitti-multiview/info.json'
Loading existing dataset 'pcd_tutorial'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


## Understanding the dataset

In FiftyOne, the KITTI Multiview dataset is a [Grouped Dataset](https://voxel51.com/docs/fiftyone/user_guide/groups.html):

In [8]:
print(dataset.media_type)

group


Each group contains three media files corresponding to the same scene - a "left" image, a "right" image, and a point cloud "pcd" file:

In [11]:
print(dataset.group_slices)

['left', 'right', 'pcd']


The default group slice for the group is "left", as we can see by inspecting the dataset:

In [12]:
dataset

Name:        pcd_tutorial
Media type:  group
Group slice: left
Num groups:  7481
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    group:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.groups.Group)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

This is the group slice whose media will appear in the sample grid when we launch the FiftyOne App:

In [15]:
session = fo.launch_app(dataset)

When we click on one of the images in the grid, it pulls up all three samples in the associated group. On the right side of FiftyOne App, we can see the point cloud data visualized in FiftyOne's $3d$ visualizer. We'll inspect these three dimensional plots interactively later on in the tutorial.

### Point clouds

In [the original dataset](https://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=3d), point-clouds, which were generated using [Velodyne LiDAR](https://velodynelidar.com/wp-content/uploads/2019/12/63-9243-Rev-E-VLP-16-User-Manual.pdf), are stored in Velodyne `.bin` files. For convenience and interoperability with most three dimensional geometry and computer vision libraries, point-clouds in FiftyOne's KITTI Multiview dataset are stored in `.pcd` files, in the [PCD file format](https://pointclouds.org/documentation/tutorials/pcd_file_format.html). 

FiftyOne's $3d$ visualizer currently supports only `.pcd` files and [PolyLines](https://voxel51.com/docs/fiftyone/user_guide/using_datasets.html#polylines-and-polygons) objects. If your point-clouds are stored in a different format, you will need to convert it first. Many solutions exist for performing these conversions. For instance, to convert from `.ply`, you can read in a file using open3d's `open3d.io.read_point_cloud()` method, and then write out the resulting object to file with the `open3d.io.write_point_cloud()` method.

**Note**: It is possible to create point-cloud only datasets in FiftyOne. All FiftyOne SDK functionality that applies to the point-clouds in grouped datasets will apply equally to point-clouds in point-cloud only datasets. However, when you launch the FiftyOne App, you will see that the sample grid does not populate!

We can inspect the point-clouds more closely using the [open3d geometry](http://www.open3d.org/docs/release/python_api/open3d.geometry.html) library.

To do this, we need to change the active group slice to the "pcd" group slice:

In [21]:
dataset.group_slice = "pcd"

Then we can select one of the samples:

In [22]:
pcd_sample = dataset.first()

Now we can verify that this sample is a point-cloud:

In [23]:
pcd_sample.media_type

'point-cloud'

**Note**: If we wanted to get the point-cloud sample associated with a given "left" or "right" sample, we could do so by getting the sample's group id with `group_id = sample.group.id`, using the `get_group()` method, `group = dataset.get_group(group_id)`, and then getting the id of the `pcd` element: `pcd_id = group["pcd"]`.

Now we can load the `.pcd` file into open3d:

In [26]:
pcd_filepath = pcd_sample["filepath"]
point_cloud = o3d.io.read_point_cloud(pcd_filepath)
print(point_cloud)

PointCloud with 115384 points.


We can also get aggregate info about the point cloud by inspecting the points in the point-cloud. For instance, we can find the minimum and maximum bounds:

In [33]:
pcd_points = np.array(point_cloud.points) ## has shape (Npoints, 3) where 2nd dim is (x, y, z) coords
min_xyz = np.amin(pcd_points, axis = 0)
max_xyz = np.amax(pcd_points, axis = 0)
print("min_xyz = ", min_xyz)
print("max_xyz = ", max_xyz)

min_xyz =  [-71.03600311 -21.10499954  -5.15999985]
max_xyz =  [73.03900146 53.79700089  2.67199993]


### $3d$ detections

In this dataset, samples have detection objects in the ground truth field:

In [34]:
pcd_sample.ground_truth

<Detections: {
    'detections': [
        <Detection: {
            'id': '636c4a0ae570df8a169571d6',
            'attributes': {},
            'tags': [],
            'label': 'Pedestrian',
            'bounding_box': [],
            'mask': None,
            'confidence': None,
            'index': None,
            'dimensions': [1.89, 0.48, 1.2],
            'location': [1.84, 1.47, 8.41],
            'rotation': [0, 0.01, 0],
        }>,
    ],
}>

Each registered detection on the point-cloud sample in a group is accompanied by a detection on the left image and on the right image. Let's see an example of this by getting the left and right images corresponding to this point-cloud sample:

In [41]:
dataset.group_slice = "pcd"
group = dataset.get_group(pcd_sample.group.id)
print(group)

{'left': <Sample: {
    'id': '636c4a34e570df8a16984680',
    'media_type': 'image',
    'filepath': '/Users/jacobmarks/fiftyone/kitti-multiview/train/left/000000.png',
    'tags': ['train'],
    'metadata': <ImageMetadata: {
        'size_bytes': 893783,
        'mime_type': 'image/png',
        'width': 1224,
        'height': 370,
        'num_channels': 3,
    }>,
    'group': <Group: {'id': '636c4a0ae570df8a169571d1', 'name': 'left'}>,
    'ground_truth': <Detections: {
        'detections': [
            <Detection: {
                'id': '636c4a0ae570df8a169571d5',
                'attributes': {},
                'tags': [],
                'label': 'Pedestrian',
                'bounding_box': [
                    0.5820261437908496,
                    0.3864864864864865,
                    0.08033496732026148,
                    0.4457297297297298,
                ],
                'mask': None,
                'confidence': None,
                'index': None,
        

Here we can see that this scene contains one identified "Pedestrian". In the left and right images, the position and shape of the box bounding the detected object is represented with a `bounding_box`. which is stored in `[<top-left-x>, <top-left-y>, <width>, <height>]` format, with units expressed relative to the image dimensions.

Detections on the point-cloud sample, on the other hand, has `dimensions`, `location`, and `rotation` properties. 

* **Dimensions**: encodes object dimensions $[height, width, length]$ in object coordinates
* **Location**: encodes the object center location [x, y, z] in point-cloud coordinates
* **Rotation**: encodes the object rotation around [x, y, z] camera axes, in $[-\pi, \pi)$

An important detail is that the coordinates of the images are *not* the same as the $(x, y)$ coordinates of the point clouds. To convert between these coordinate systems, as we will do later on, we need information about the orientations of the lidar sensor and cameras used. For the KITTI Multiview dataset, this is encoded in a set of calibration matrices for each scene, which are stored in a text file in the `kitti-multiview/train/calib` and `kitti-multiview/test/calib` directories. These calibration files are not included in the quickstart groups dataset.

### Labels in the data

Before proceding to the model, it's also important to get a sense for what objects are present in our dataset. We can aggregate counts for all of the detections by label with the `count_values()` method:

In [50]:
dataset.count_values("ground_truth.detections.label")

{'Van': 2914,
 'Tram': 511,
 'Pedestrian': 4487,
 'DontCare': 11295,
 'Misc': 973,
 'Car': 28742,
 'Truck': 1094,
 'Cyclist': 1627,
 'Person_sitting': 222}

All of the scenes present in this dataset were captured on or near roads, hence people being labeled as "Pedestrian" and "Cyclist". The label "DontCare" encompasses all detected objects that do not fall into one of the other listed classes. 

In what follows, we will restrict our attention to objects in the "Car", "Pedestrian", and "Cyclist" classes.

In [2]:
dataset = foz.load_zoo_dataset("kitti-multiview")

Parsing dataset metadata
Found 22443 samples
Parsing dataset metadata
Found 22554 samples
Dataset info written to '/Users/jacobmarks/fiftyone/kitti-multiview/info.json'
Loading 'kitti-multiview' split 'train'
Importing samples...
 100% |█████████████| 22443/22443 [1.3s elapsed, 0s remaining, 17.9K samples/s]         
Migrating dataset 'kitti-multiview' to v0.18.0
Import complete
Loading 'kitti-multiview' split 'test'
Importing samples...
 100% |█████████████| 22554/22554 [329.9ms elapsed, 0s remaining, 68.4K samples/s]     
Migrating dataset '2022.11.25.14.38.14' to v0.18.0
Import complete
Dataset 'kitti-multiview' created


In [3]:
dataset.group_slice = "pcd"
pcd_sample = dataset.first()

In [4]:
train_view = dataset.match_tags("train")

In [6]:
dataset.group_slice = "left"
session = fo.launch_app(dataset)

In [13]:
dataset = dataset.match_tags("train")[:200].clone()

view = dataset[0:100]
### Get rid of right slice because we don't use it at all
view.group_slice = "right"
for group in view.iter_groups():
    r = group['right'].id
    dataset.delete_samples(r)

## Generate feature maps

In [14]:
# Front side (of vehicle) Point Cloud boundary for BEV
min_bound = (0, -25, -2.73)
max_bound = (50, 25, 1.27)
bev_width = 608
bev_height = 608

In [23]:
dataset.group_slice = "pcd"

In [24]:
fou3d.compute_birds_eye_view_maps(dataset, bev_width, bev_height, min_bound = min_bound, max_bound = max_bound)

Parsing samples...
 100% |█████████████████| 200/200 [17.9s elapsed, 0s remaining, 11.3 samples/s]      


In [25]:
class_list = ["Car", "Pedestrian", "Cyclist"]

In [26]:
train_view.group_slice = "pcd"
height = F("dimensions")[0]
exp = F("ground_truth.detections").map(height)
heights_dict = {}
for c in class_list:
    heights_dict[c] = train_view.filter_labels(
    "ground_truth", F("label") == c).mean(exp)

In [27]:
# session = fo.launch_app(train_view, auto = False)

In [28]:
# session.url

## Apply Model

In [29]:
conf_thresh, nms_thresh = 0.9, 0.3
img_size = bev_height
args = {"img_size": img_size, "conf_thresh": conf_thresh, "nms_thresh":nms_thresh}
model = foz.load_zoo_model("complex-yolo-v3-torch", args = args)

In [30]:
model.set_class_heights(heights_dict)

In [31]:
dataset.group_slice = "bev"

In [32]:
import fiftyone.utils.complex_yolo as foucy
foucy.apply_model(model, dataset, feature_map_field = 'feature_map_filepath', pcd_group_slice = 'pcd', min_bound = min_bound, max_bound = max_bound)

 100% |██████████████████████████████████████████████████████████████████████████████████████| 200/200 [59.4s elapsed, 0s remaining, 3.4 it/s]      


## Transform coordinates

In [36]:
def get_calib_path(sample):
    sample_path = sample.filepath.split("/")
    calib_path = sample_path.copy()
    calib_path[-2] = "calib"
    calib_path = "/".join(calib_path)
    calib_path = calib_path[:-3] + "txt"
    return calib_path

In [119]:
def get_left_from_pcd(pcd_sample):
    dataset = pcd_sample._dataset
    dataset.group_slice = "pcd"
    group = dataset.get_group(pcd_sample.group.id)
    left_id = group["left"].id
    dataset.group_slice = "left"
    left_sample = dataset[left_id]
    return left_sample

In [120]:
def get_corners3d(h, w, l, R, t):
    corners3d = np.array(
        [
            [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2],
            [0, 0, 0, 0, -h, -h, -h, -h],
            [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2],
        ]
    )
    return R @ corners3d + t[:, np.newaxis]

In [184]:
def get_img2d_shape(img_sample):
    if "metadata" in sample:
        return img_sample.metadata.width, img_sample.metadata.height
    else:
        h, w = cv2.imread(img_sample.filepath).shape[:2]
        return w, h

In [190]:
def corners3d_to_img_boxes(P, corners3d, img2d_shape):
    """
    :param corners3d: (N, 8, 3) corners in rect coordinate
    :return: boxes: (N, 4) [x1, y1, x2, y2] in img2d coords
    """
    corners3d = corners3d
    num_boxes = corners3d.shape[0]
    corners3d_hom = np.concatenate((corners3d, np.ones((num_boxes, 8, 1))), axis=2)  # (N, 8, 4)
    img_pts = np.matmul(corners3d_hom, P.T)  # (N, 8, 3)

    x, y = img_pts[:, :, 0] / img_pts[:, :, 2], img_pts[:, :, 1] / img_pts[:, :, 2]
    x1, y1 = np.min(x, axis=1).reshape(-1, 1), np.min(y, axis=1).reshape(-1, 1)
    x2, y2 = np.max(x, axis=1).reshape(-1, 1), np.max(y, axis=1).reshape(-1, 1)
    
    boxes = np.concatenate((x1, y1, x2, y2), axis=1)
    
    boxes[:, 0] = np.clip(boxes[:, 0], 0, img2d_shape[0] - 1)
    boxes[:, 1] = np.clip(boxes[:, 1], 0, img2d_shape[1] - 1)
    boxes[:, 2] = np.clip(boxes[:, 2], 0, img2d_shape[0] - 1)
    boxes[:, 3] = np.clip(boxes[:, 3], 0, img2d_shape[1] - 1)
    
    boxes[:, 0]/=img2d_shape[0]
    boxes[:, 2]/=img2d_shape[0]
    boxes[:, 1]/=img2d_shape[1]
    boxes[:, 3]/=img2d_shape[1]
    
    boxes[:, 2] -= boxes[:, 0]
    boxes[:, 3] -= boxes[:, 1]
    
    return boxes

In [217]:
def add_pcd_sample_detections_to_image(pcd_sample):
    if "predictions" not in pcd_sample or len(pcd_sample.predictions.detections) == 0:
        return
    calib_mats = fouk._load_calibration_matrices(get_calib_path(pcd_sample))
    P = calib_mats["P2"]
    
    left_sample = get_left_from_pcd(pcd_sample)
    img2d_shape = get_img2d_shape(left_sample)
    
    corners3d = []
    detections3d = pcd_sample.predictions.detections
    
    for det3d in detections3d:
        h, w, l = det3d["dimensions"]
        t = np.array(det3d["location"])
        R = fouk._roty(det3d["rotation"][1])
        corners3d.append(get_corners3d(h, w, l, R, t).T)
    corners3d = np.array(corners3d)
    bboxes = corners3d_to_img_boxes(P, corners3d, img2d_shape)
    
    detections2d = []
    for i, det3d in enumerate(detections3d):
        new_det2d = fo.Detection(
            label=det3d.label,
            bounding_box=bboxes[i],
            confidence=det3d.confidence,
            alpha = det3d.alpha
        )
        detections2d.append(new_det2d)
    
    left_sample["predictions"] = fo.Detections(detections=detections2d)
    left_sample.save()
        

In [222]:
def project_detections_to_images(pcd_samples):
    for pcd_sample in pcd_samples:
        add_pcd_sample_detections_to_image(pcd_sample)

In [224]:
dataset.group_slice = "pcd"
pcd_samples = dataset.select_group_slices("pcd")

In [225]:
project_detections_to_images(pcd_samples)

## Evaluate detection results

In [236]:
dataset.group_slice = "left"
eval_view_2d = dataset.filter_labels(
        "ground_truth", 
        F("label").is_in(class_list)
)

results_2d = eval_view_2d.evaluate_detections(
    "predictions",
    iou = 0.5,
    compute_mAP=True,
)

In [249]:
dataset.group_slice = "pcd"
eval_view_3d = dataset.filter_labels(
        "ground_truth", 
        F("label").is_in(class_list)
)

results_3d = eval_view_3d.evaluate_detections(
    "predictions",
    iou = 0.3,
    compute_mAP=True,
)

Evaluating detections...
 100% |█████████████████| 200/200 [649.5ms elapsed, 0s remaining, 307.9 samples/s]      
Performing IoU sweep...
 100% |█████████████████| 200/200 [955.2ms elapsed, 0s remaining, 196.5 samples/s]      


In [250]:
results_2d.print_report()

              precision    recall  f1-score   support

         Car       0.72      0.70      0.71       733
     Cyclist       0.60      0.54      0.57        46
  Pedestrian       0.10      0.11      0.10       104

   micro avg       0.63      0.63      0.63       883
   macro avg       0.47      0.45      0.46       883
weighted avg       0.64      0.63      0.63       883



In [248]:
results_3d.print_report()

              precision    recall  f1-score   support

         Car       0.88      0.86      0.87       733
     Cyclist       0.62      0.57      0.59        46
  Pedestrian       0.34      0.37      0.35       104

   micro avg       0.80      0.79      0.79       883
   macro avg       0.61      0.60      0.60       883
weighted avg       0.80      0.79      0.80       883



## To do

* Add detections to BEV - ground truth and predictions
* Add visualizations along the way
    * BEV - only in front of car --> bounds
* Description of the model
* RGB BEV encoding
    * And how the compute_BEV function stores its data, plus WHY
* Limitations of the model - no height info --> used average height by class
* Add in average $z$ by class
* Talk about how the model could be improved with sensor fusion
* Different IoU thresholds in $2d$ vs $3d$ because overlap gets smaller as the number of dimensions increases...
* Mention setting group slice whenever we want to access a particular group, and how we can't visualize in the app if the group slice is set to pcd
* Mention right clicking and moving the mouse/mousepad to drag position in the $3d$ viewer